(categorical-data)=
# 범주형 데이터

## 소개

이 장에서는 범주형 변수, 즉 고정되고 알려진 가능한 값 집합을 가진 변수를 다루는 방법을 소개합니다. 이 장은 **pandas** [문서](https://pandas.pydata.org/)에 크게 의존합니다.


In [ ]:
# 셀 제거
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# 플롯 설정
plt.style.use("https://github.com/aeturrell/python4DS/raw/main/plot_style.txt")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

### 전제 조건

이 장에서는 **pandas** 데이터 분석 패키지를 사용합니다.

## 범주 데이터 유형

파이썬의 모든 것에는 유형이 있으며 **pandas** 데이터 프레임 열의 데이터도 마찬가지입니다. 숫자나 문자열에 더 익숙할 수 있지만 범주형 데이터에는 `Categorical`이라는 특수 데이터 유형도 있습니다. (적절한 경우) 범주형 변수를 사용하면 몇 가지 이점이 있습니다.

- 범주의 요소가 없는 경우에도 추적할 수 있으며, 이는 때때로 요소가 있을 때만큼 흥미로울 수 있습니다(특정 학교 출신의 학생이 대학에 가지 않는다는 사실을 발견했다고 상상해 보십시오).
- 다른 방식으로 동일한 정보를 인코딩하는 것보다 컴퓨터 메모리를 훨씬 적게 사용할 수 있습니다.
- 잠재적인 '더미 변수'로 인식될 모델링 패키지나 불연속 값으로 처리할 플로팅 패키지와 함께 효율적으로 사용할 수 있습니다.
- 순서를 지정할 수 있습니다(예: "중립", "동의", "매우 동의").

**pandas** 열의 모든 범주형 데이터 값은 지정된 범주에 있거나 `np.nan` 값을 갖습니다.

## 범주형 데이터 만들기

데이터의 범주형 열을 만들어 보겠습니다.

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame({"A": ["a", "b", "c", "a"]})

df["A"] = df["A"].astype("category")
df["A"]

표시된 계열 하단에 추가 정보가 표시됩니다. 이 열이 범주형 열 유형일 뿐만 아니라 'a', 'b', 'c'의 세 가지 값을 갖는다는 것을 알 수 있습니다.

`pd.cut()`과 같은 특수 함수를 사용하여 데이터를 불연속적인 구간으로 그룹화할 수도 있습니다. 다음은 범주에 대한 레이블을 직접 지정하는 예입니다.

In [ ]:
df = pd.DataFrame({"value": np.random.randint(0, 100, 20)})
labels = [f"{i} - {i+9}" for i in range(0, 100, 10)]
df["group"] = pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)
df.head()

위 예에서 `group` 열은 범주형 유형입니다.

범주형 변수를 만드는 또 다른 방법은 `pd.Categorical()` 함수를 직접 사용하는 것입니다.

In [ ]:
raw_cat = pd.Categorical(
    ["a", "b", "c", "a", "d", "a", "c"], categories=["b", "c", "d"]
)
raw_cat

그런 다음 이것을 데이터 프레임에 입력할 수 있습니다.

In [ ]:
df = pd.DataFrame(raw_cat, columns=["cat_type"])
df["cat_type"]

지정한 범주에 *없는* 값에 대해 NaN이 나타납니다. 이에 대한 자세한 내용은 {ref}`missing-values`에서 확인할 수 있습니다.

순서가 지정된 범주를 만들 수도 있습니다.

In [ ]:
ordered_cat = pd.Categorical(
    ["a", "b", "c", "a", "d", "a", "c"],
    categories=["a", "b", "c", "d"],
    ordered=True,
)
ordered_cat

## 범주 작업

범주형 데이터에는 `categories` 및 `ordered` 속성이 있습니다. 이러한 속성은 각각 가능한 값과 순서 지정 여부를 나열합니다. 이러한 속성은 `.cat.categories` 및 `.cat.ordered`로 노출됩니다. 범주 및 순서를 수동으로 지정하지 않으면 전달된 인수에서 유추됩니다.

몇 가지 예를 살펴보겠습니다.

In [ ]:
df["cat_type"].cat.categories

In [ ]:
df["cat_type"].cat.ordered

범주형 데이터가 정렬된 경우(즉, `.cat.ordered == True`), 범주 순서에 의미가 있으며 특정 작업이 가능합니다. 값을 정렬하고(`.sort_values()`), `.min` 및 `.max`를 적용할 수 있습니다.

### 범주 이름 바꾸기

범주 이름 바꾸기는 `rename_categories()` 메서드(목록 또는 사전과 함께 작동)를 통해 수행됩니다.

In [ ]:
df["cat_type"] = df["cat_type"].cat.rename_categories(["alpha", "beta", "gamma"])

범주를 추가하려는 상황에 자주 직면하게 될 것입니다. `.add_categories()`를 사용하여 이 작업을 수행할 수 있습니다.

In [ ]:
df["cat_type"] = df["cat_type"].cat.add_categories(["delta"])
df["cat_type"]

마찬가지로 `.remove_categories()` 함수와 `.remove_unused_categories()` 함수가 있습니다. `.set_categories`는 범주를 한 번에 추가하고 제거합니다. 설정된 범주의 좋은 속성 중 하나는 다음과 같습니다. 하지만 cat(egory) 함수를 호출하기 전에 `df["columnname"].cat`를 수행해야 한다는 것을 기억하십시오.

## 범주에 대한 연산

언급했듯이 정렬된 범주는 이미 일부 작업을 거칩니다. 그러나 모든 범주 집합에 대해 작동하는 일부 작업이 있습니다. 아마도 가장 유용한 것은 `value_counts()`일 것입니다.

In [ ]:
df["cat_type"].value_counts()

'델타'가 전혀 나타나지 않더라도 개수(0)를 얻습니다. 이러한 결측값 추적은 매우 유용할 수 있습니다.

`mode()`는 또 다른 것입니다.

In [ ]:
df["cat_type"].mode()

그리고 범주형 열이 우연히 작업을 수행할 수 있는 *요소*로 구성된 경우 동일한 작업이 계속 작동합니다. 예를 들어,

In [ ]:
time_df = pd.DataFrame(
    pd.Series(pd.date_range("2015/05/01", periods=5, freq="M"), dtype="category"),
    columns=["datetime"],
)
time_df

In [ ]:
time_df["datetime"].dt.month

마지막으로 범주형 열의 실제 데이터 유형을 코드로 변환해야 하는 경우 `.cat.codes`를 사용하여 각 값에 대한 고유 코드를 얻을 수 있습니다.

In [ ]:
time_df["datetime"].cat.codes